## Convertir ficheros CSV a JSON (utilizando Pandas) para cargar en Elasticsearch con procedimiento bulk

~~~~
pandas.read_csv(filepath_or_buffer, sep=NoDefault.no_default, delimiter=None, header='infer', names=NoDefault.no_default, index_col=None, usecols=None, squeeze=False, prefix=NoDefault.no_default, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal='.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, encoding_errors='strict', dialect=None, error_bad_lines=None, warn_bad_lines=None, on_bad_lines=None, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None, storage_options=None)
~~~~

In [3]:
import pandas as pd
df = pd.read_csv("ratings_test_header.csv", infer_datetime_format=True, delimiter=',')
df.head()

userId  movieId  rating       fecha
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [4]:
df.dtypes

userId       int64
movieId      int64
rating     float64
fecha        int64
dtype: object

In [5]:
df['fecha']=pd.to_datetime(df["fecha"], unit='s')

In [6]:
df.dtypes

userId              int64
movieId             int64
rating            float64
fecha      datetime64[ns]
dtype: object

In [7]:
df.head()

userId  movieId  rating               fecha
0       1       31     2.5 2009-12-14 02:52:24
1       1     1029     3.0 2009-12-14 02:52:59
2       1     1061     3.0 2009-12-14 02:53:02
3       1     1129     2.0 2009-12-14 02:53:05
4       1     1172     4.0 2009-12-14 02:53:25

In [8]:
format1 = "%Y-%m-%d %H:%M:%S"
format2 = "%Y-%m-%dT%H:%M:%S"

df['fecha'] = df['fecha'].dt.strftime('%Y-%m-%dT%H:%M:%S')

df.head()

userId  movieId  rating                fecha
0       1       31     2.5  2009-12-14T02:52:24
1       1     1029     3.0  2009-12-14T02:52:59
2       1     1061     3.0  2009-12-14T02:53:02
3       1     1129     2.0  2009-12-14T02:53:05
4       1     1172     4.0  2009-12-14T02:53:25

In [9]:
df['fecha'] = df['fecha'].astype(str)
df.head()

userId  movieId  rating                fecha
0       1       31     2.5  2009-12-14T02:52:24
1       1     1029     3.0  2009-12-14T02:52:59
2       1     1061     3.0  2009-12-14T02:53:02
3       1     1129     2.0  2009-12-14T02:53:05
4       1     1172     4.0  2009-12-14T02:53:25

In [10]:
df.dtypes

userId       int64
movieId      int64
rating     float64
fecha       object
dtype: object

~~~
DataFrame.to_json(path_or_buf=None, orient=None, date_format=None, double_precision=10, force_ascii=True, date_unit='ms', default_handler=None, lines=False, compression='infer', index=True, indent=None, storage_options=None)
~~~

In [11]:
df.to_json(path_or_buf='ratings_test.json', orient='records', lines=True)

In [12]:
finput = open("ratings_test.json", "r")
foutput = open("ratings_test_bulk.json", "w")

linea = finput.readline()

while linea:
    foutput.write("{ \"index\":{} }\n")
    foutput.write(linea)
    linea = finput.readline()
foutput.write("\n")

finput.close()
foutput.close()                  